Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# Set Cover problem

See: https://en.wikipedia.org/wiki/Set_cover_problem

In [170]:
from random import random, seed
from itertools import product
import numpy as np

from icecream import ic

## Reproducible Initialization

If you want to get reproducible results, use `rng` (and restart the kernel); for non-reproducible ones, use `np.random`.

In [171]:
UNIVERSE_SIZE = 1000
NUM_SETS = 100
DENSITY = 0.2 # 20% of the elements are in each set

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)])) #rng is the random number generator, so we can use it to generate random numbers

In [172]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY #generate the sets
for s in range(UNIVERSE_SIZE): #ensure that each element is in at least one set
    if not np.any(SETS[:, s]): #if the element is not in any set
        SETS[np.random.randint(NUM_SETS), s] = True #add it to a random set
COSTS = np.power(SETS.sum(axis=1), 1.1) #compute the costs of each set #the costs? -> the number of elements in the set to the power of 1.1

## Helper Functions

In [173]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution])) #check if all elements are in at least one set


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum() #return the sum of the costs of the sets in the solution 

## Have Fun!

In [174]:
# A dumb solution of "all" sets
solution = np.full(NUM_SETS, True) 
valid(solution), cost(solution) 

#(True, 841030176.2660773) #the solution is valid and the cost is 841030176.2660773

(True, 34023.83523034387)

In [175]:
# A random solution with random 50% of the sets
solution = rng.random(NUM_SETS) < .5 #generate a random solution. Each set is in the solution with 50% probability
print("solution:", solution)
valid(solution), cost(solution)

#(True, 420515088.13303865) #the solution is valid and the cost is 420515088.13303865

solution: [False  True  True False False False False False False False False False
 False False  True  True  True  True False False  True False False  True
  True  True  True False False  True  True  True False False  True  True
 False  True False False  True  True False False  True False  True False
 False  True  True  True  True False  True  True False False False  True
  True False False False False False  True  True  True  True False False
  True  True  True False False  True  True  True False False False  True
  True False False False False  True False  True False  True  True False
 False False  True  True]


(True, 16225.557691518967)

In [176]:
#Solve efficiently these instances
# Instance	Universe size	Num sets	Density
# 1	100	10	.2 #is possible to resolve it by brute force? -> 
# 2	1,000	100	.2
# 3	10,000	1,000	2
# 4	100,000	10,000	.1
# 5	100,000	10,000	
# 6	100,000	10,000	.3 


In [177]:
# SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY #generate the sets
# for s in range(UNIVERSE_SIZE): #ensure that each element is in at least one set
#     if not np.any(SETS[:, s]): #if the element is not in any set
#         SETS[np.random.randint(NUM_SETS), s] = True #add it to a random set
# COSTS = np.power(SETS.sum(axis=1), 1.1) #compute the costs of each set #the costs? -> the number of elements in the set to the power of 1.1
#COSTS
#SETS #why we have false and true values? -> the elements are in the set or not

#-------------
# we have for example 1 2 3 in the universe of elements and we have 3 sets with the elements 1 2, 1 3, 2 3
# the first set (which includes true/false repeated for the number of elements in the elements universe) will be [True, True, False] because the elements 1 and 2 are in the set and the element 3 is not in the set. Then we have [True, False, True] and [False, True, True].
# to represent the universe with the sets we can see the first set and see where is false. then we can see the second set and see if where before was false now is true. if the 'or' operation (union) before the first set and second set computes a set of all True values then the solution is valid. otherwise i continue with the third set. 

# we can do:
# 1 Start with an empty solution (no sets selected).
# 2 While there are uncovered elements in the universe:
# Select the set that covers the largest number of uncovered elements.
# Add that set to the solution.
# 3 Continue until the union of selected sets covers the entire universe.

#A solution is valid only if, after selecting some sets, the union of those sets contains only True values.


In [178]:
def greedy_set_cover(SETS, UNIVERSE_SIZE):
    # SETS is a NumPy array of shape (NUM_SETS, UNIVERSE_SIZE) where each row represents a set
    #print("sets are:", SETS)
    uncovered = np.ones(UNIVERSE_SIZE, dtype=bool)  # Track which elements are uncovered (all True initially) #np.ones ? -> create an array of ones
    selected_sets = []  # Store the indices of selected sets #empty list initially, then we will add the indices of the selected sets where 
    
    while np.any(uncovered):  # While there are still uncovered elements #np.any() here used to check whether any set covers a specific element or to verify if all elements of the universe are covered by the selected sets.
        # Choose the set that covers the most uncovered elements
        best_set_idx = None
        best_coverage = 0
        
        for i, set_i in enumerate(SETS): #enumerate? -> returns an enumerate object. It contains the index and value of all the items in the set as pairs. example: enumerate(sets) -> [(0, [True, True, False]), (1, [True, False, True]), (2, [False, True, True])]
            coverage = np.sum(uncovered & set_i)  # Count how many uncovered elements this set can cover #example -> uncovered = [True, True, True], set_i = [True, True, False], uncovered & set_i = [True, True, False] -> np.sum([True, True, False]) = 2
            if coverage > best_coverage: 
                best_coverage = coverage 
                best_set_idx = i #store the index of the set that covers the most uncovered elements
        
        if best_set_idx is None: #if there is no set that covers any remaining elements 
            # If no set can cover any remaining elements, it's impossible to cover the universe
            raise ValueError("No valid set cover found!")
        
        # Add the best set to the solution and update the uncovered elements
        selected_sets.append(best_set_idx) #add the index of the best set to the selected sets
        uncovered = uncovered & ~SETS[best_set_idx]  # Mark elements covered by this set as covered (False) #update the uncovered elements by removing the elements that are covered by the best set #example -> uncovered = [True, True, True], SETS[best_set_idx] = [True, True, False], ~SETS[best_set_idx] = [False, False, True], uncovered & ~SETS[best_set_idx] = [True, True, False]
    print("\nselected sets:", SETS[selected_sets]) 
    return selected_sets #return the indices of the selected sets. if we want to return the matrix with the selected sets we can use SETS[selected_sets]

#we are not sure that the solution is valid. we have a probabilstic approach, taking the set that covers the most uncovered elements and we 'hope' that the union of the selected sets will cover the entire universe.



In [179]:
# Compute the solution using the greedy algorithm
solution_indices = greedy_set_cover(SETS, UNIVERSE_SIZE)

# Check if the solution is valid and compute its cost
print("\nSelected sets - indexes:", solution_indices)
print("\nSelected sets - len indexes:", len(solution_indices))
is_valid = valid(solution_indices)
solution_cost = cost(solution_indices)

print(f"\nIs the solution valid? {is_valid}")
print(f"Total cost of the solution: {solution_cost}")


selected sets: [[False False  True ...  True False False]
 [False False False ... False False  True]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False  True]]

Selected sets - indexes: [49, 61, 14, 41, 40, 15, 2, 87, 24, 42, 68, 86, 35, 51, 9, 67, 4, 47]

Selected sets - len indexes: 18

Is the solution valid? True
Total cost of the solution: 6300.920399618765
